# Combinar conjuntos de datos: Merge y join

Una característica esencial que ofrece Pandas son sus operaciones join y merge de alto rendimiento en memoria.
Si alguna vez has trabajado con bases de datos, deberías estar familiarizado con este tipo de interacción de datos.
La interfaz principal para esto es la función ``pd.merge``, y veremos algunos ejemplos de cómo puede funcionar en la práctica.

Por conveniencia, empezaremos redefiniendo la funcionalidad ``display()`` de la sección anterior:

In [3]:
import pandas as pd
import numpy as np

class display(object):
    """Mostrar la representación HTML de varios objetos"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

## Álgebra Relacional

El comportamiento implementado en ``pd.merge()`` es un subconjunto de lo que se conoce como *álgebra relacional*, que es un conjunto formal de reglas para manipular datos relacionales, y forma la base conceptual de las operaciones disponibles en la mayoría de las bases de datos.
El punto fuerte del enfoque del álgebra relacional es que propone varias operaciones primitivas, que se convierten en los bloques de construcción de operaciones más complicadas sobre cualquier conjunto de datos.
Con este léxico de operaciones fundamentales implementado de forma eficiente en una base de datos u otro programa, se puede realizar una amplia gama de operaciones compuestas bastante complicadas.

Pandas implementa varios de estos bloques de construcción fundamentales en la función ``pd.merge()`` y el método relacionado ``join()`` de ``Series`` y ``Dataframe``.
Como veremos, estos métodos permiten enlazar datos de diferentes fuentes.

>>> ## Categorías de uniones

**LOS DATAFRAMES PUEDEN UNIRSE, SI TIENEN:**
- **COLUMNAS** en común = **`pd.merge()`** busca las columnas que coinciden y los une.
- **ÍNDICES** en común = **`pd.concat()`** une por las índices que coinciden. Y **`.join()`** también une por índices.
  
La función ``pd.merge()`` implementa varios tipos de uniones: 
1. Uniones **uno a uno**, 
2. Uniones **muchos a uno** y 
3. Uniones **muchos a muchos**.

**- `inner`** > Por defecto, cuando utilizamos `pd.merge()`, opera con la función `inner`, es decir, uno según la intersección de ambos DataFrames, si quiero lo contrario, DEBO ESPECIFICAR `how=outer`.

**SINTAXIS** = `pd.merge(df3, df4, how='outer')`


>> ### **1. Uniones UNO a UNO**

- Buscará las columnas que coinciden y unirá los DataFrames por ahí.
- Como cada columna puede tener los mismos datos que la otra, pero diferentes índices, se guiará por el orden de la columna del primer DF que ponga entre paréntesis. Si quiero al revés, cambio el orden de los DataFrames al hacer `pd.merge()`

In [1]:
import pandas as pd
import numpy as np

In [4]:
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})
display('df1', 'df2')

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014


Para combinar esta información en un único ``DataFrame``, podemos utilizar la función ``pd.merge()``.

* En el siguiente ejemplo, busco la columna coincidente que sería *"employee"* y unió los DF por ahí. Teniendo en cuenta el `df1`, si quiero al revés pongo `df2` primero. 

In [6]:
df3 = pd.merge(df1, df2)
df3

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


- **La función ``pd.merge()`` reconoce que cada ``DataFrame`` tiene una columna "empleado", y une automáticamente utilizando esta columna como clave.**

- Además, hay que tener en cuenta que la fusión en general **descarta el índice**, excepto en el caso especial de las fusiones por índice, mediante: ``left_index`` y ``right_index``

>> ### **2. Uniones MUCHOS a UNO**

Las uniones múltiples son uniones en las que **una de las dos columnas clave contiene entradas duplicadas**.
En este caso, el ``DataFrame`` resultante conservará las entradas duplicadas.

Considere el siguiente ejemplo de unión muchos-a-uno:

In [4]:
df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR',],
                    'supervisor': ['Carly', 'Guido', 'Steve']})
display('df3', 'df4', 'pd.merge(df3, df4)')

df3
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014

df4
         group supervisor
0   Accounting      Carly
1  Engineering      Guido
2           HR      Steve

pd.merge(df3, df4)
  employee        group  hire_date supervisor
0      Bob   Accounting       2008      Carly
1     Jake  Engineering       2012      Guido
2     Lisa  Engineering       2004      Guido
3      Sue           HR       2014      Steve

El ``DataFrame`` resultante tiene una columna adicional con la información del "supervisor", donde la información se repite en uno o más lugares según lo requieran las entradas.

>> ### **3. Uniones MUCHOS a MUCHOS**

In [7]:
df5 = pd.DataFrame({'group': ['Accounting', 'Accounting',
                              'Engineering', 'Engineering', 'HR', 'HR'],
                    'skills': ['math', 'spreadsheets', 'coding', 'linux',
                               'spreadsheets', 'organization']})
display('df1', 'df5', "pd.merge(df1, df5)")

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,group,skills
0,Accounting,math
1,Accounting,spreadsheets
2,Engineering,coding
3,Engineering,linux
4,HR,spreadsheets


Estos tres tipos de uniones se pueden utilizar con otras herramientas de Pandas para implementar una amplia gama de funcionalidades.

En la siguiente sección consideraremos algunas de las opciones proporcionadas por ``pd.merge()`` que permiten ajustar el funcionamiento de las operaciones de unión.

>>> ## **Especificación de la CLAVE de FUSIÓN**
Ya hemos visto el comportamiento por defecto de ``pd.merge()``: busca uno o más nombres de columna que coincidan entre las dos entradas y los utiliza como clave.

Sin embargo, a menudo los nombres de las columnas no coinciden tan bien, y ``pd.merge()`` proporciona una variedad de opciones para manejar esto.

>> ### **La palabra clave ``on=``**

Solo funciona si los 2 DataFrames tienen en común el MISMO NOMBRE DE LA COLUMNA. 

Podría funcionar si por ejemplo el `pd.merge()` automaticamente los une por `'employee'` que es la primera columna que encuentra, y yo quiero que los una por `'group'` (si los dos df tuvieran también esa columna en común, que en el ejemplo NO la tienen por lo que daría error)

In [6]:
display('df1', 'df2', "pd.merge(df1, df2, on='employee')")

df1
  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

df2
  employee  hire_date
0     Lisa       2004
1      Bob       2008
2     Jake       2012
3      Sue       2014

pd.merge(df1, df2, on='employee')
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014

>> ### **Las palabras clave ``left_on`` y ``right_on``**

En ocasiones, es posible que desee combinar **dos conjuntos de datos** con **nombres de columna diferentes**; por ejemplo, podemos tener un conjunto de datos en el que el nombre del empleado esté etiquetado como "nombre" en lugar de "empleado".

Entonces, tengo que:
1. Declarar el **orden** de los DataFrames.
2. `left_on=` > Especificar el nombre de la columna que declaré primero, a la izquierda.
3. `right_on=` > Especificar el nombre de la columna que declaré segundo, a la derecha.

- Si tuvieran **distintos orden de índice las filas** dentro de cada columna, respetará el orden de la primer columna declarada, es decir, la que pongo a la izquierda `left_on`. 

In [12]:
df3 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'salary': [70000, 80000, 120000, 90000]})
display('df1', 'df3', 'pd.merge(df1, df3, left_on="employee", right_on="name")')

df1
  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

df3
   name  salary
0   Bob   70000
1  Jake   80000
2  Lisa  120000
3   Sue   90000

pd.merge(df1, df3, left_on="employee", right_on="name")
  employee        group  name  salary
0      Bob   Accounting   Bob   70000
1     Jake  Engineering  Jake   80000
2     Lisa  Engineering  Lisa  120000
3      Sue           HR   Sue   90000

> #### `pd.merge()` + `.drop()` = Eliminar columna redundante luego del .merge( )
- El resultado va a tener una columna redundante, SI LOS DATOS SON IGUALES, que **podemos eliminar** si lo deseamos, por ejemplo, utilizando el método **``drop()`` de ``DataFrame``**.
- Te guarda la columna, porque los datos pueden ser diferentes, entonces revisar eso antes de eliminar la de la derecha. 
- Para que NO dé ERROR hay que poner obligatoriamente el axis = `.drop('...', axis=1)`

> - **OPCIÓN UNO** = Hacer todo de una vez:

In [17]:
pd.merge(df1, df3, left_on="employee", right_on="name").drop('name', axis=1)

,employee,group,salary
0,Bob,Accounting,70000
1,Jake,Engineering,80000
2,Lisa,Engineering,120000
3,Sue,HR,90000


> - **OPCIÓN DOS** = Guardar el merge en una variable y luego hacer el drop

In [52]:
mergeado = pd.merge(df1, df3, left_on="employee", right_on="name")
mergeado = mergeado.drop('name', axis=1)
mergeado

,employee,group,salary
0,Bob,Accounting,70000
1,Jake,Engineering,80000
2,Lisa,Engineering,120000
3,Sue,HR,90000


>> ### **Las palabras clave ``left_index`` y ``right_index``**

A veces, en lugar de **fusionar** por columna, se desea hacerlo por **ÍNDICE**.

> #### **`.set_index()`:**
Antes los df tenían índice implícito, y ahora **antes de fusionarlos**, digo que los nuevos df van a tener como índice los datos de una columna, para luego unirlos por ese índice que tendrán en común:

In [24]:
df1a = df1.set_index('employee')
df2a = df2.set_index('employee')
display('df1','df2','df1a', 'df2a')

df1
  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

df2
  employee  hire_date
0     Lisa       2004
1      Bob       2008
2     Jake       2012
3      Sue       2014

df1a
                group
employee             
Bob        Accounting
Jake      Engineering
Lisa      Engineering
Sue                HR

df2a
          hire_date
employee           
Lisa           2004
Bob            2008
Jake           2012
Sue            2014

Puedes utilizar el índice como clave para la fusión especificando los indicadores ``left_index`` y/o ``right_index`` en ``pd.merge()``:

In [37]:
display('df1a', 'df2a',
        "pd.merge(df1a, df2a, left_index=True, right_index=True)")

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR
,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012


>> #### **`.join()` = Fusión por INDICES**
- **SINTAXIS = `df1a.join(df2a)`**

Por comodidad, los ``DataFrame`` implementan el método ``join()``, que realiza una fusión que por defecto une los índices:

In [26]:
display('df1a', 'df2a', 'df1a.join(df2a)')

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR
,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012


>> #### **`pd.merge()` por INDICES + COLUMNAS**
Si quieres mezclar índices y columnas, puedes combinar:
- ``left_index`` con ``right_on`` o 
- ``left_on`` con ``right_index`` 

In [39]:
display('df1a', 'df3', "pd.merge(df1a, df3, left_index=True, right_on='name')")

df1a
                group
employee             
Bob        Accounting
Jake      Engineering
Lisa      Engineering
Sue                HR

df3
   name  salary
0   Bob   70000
1  Jake   80000
2  Lisa  120000
3   Sue   90000

pd.merge(df1a, df3, left_index=True, right_on='name')
         group  name  salary
0   Accounting   Bob   70000
1  Engineering  Jake   80000
2  Engineering  Lisa  120000
3           HR   Sue   90000

Todas estas opciones también funcionan con múltiples índices y/o múltiples columnas; la interfaz para este comportamiento es muy intuitiva.
Para más información sobre esto, consulte la sección ["Merge, Join, and Concatenate"](http://pandas.pydata.org/pandas-docs/stable/merging.html) de la documentación de Pandas.

>> ## **Especificación de la aritmética de conjuntos para uniones**
Esto ocurre cuando un **valor/dato** aparece en una columna clave pero NO en la otra. Veamos este ejemplo:

In [40]:
df6 = pd.DataFrame({'name': ['Peter', 'Paul', 'Mary'],
                    'food': ['fish', 'beans', 'bread']},
                   columns=['name', 'food'])
df7 = pd.DataFrame({'name': ['Mary', 'Joseph'],
                    'drink': ['wine', 'beer']},
                   columns=['name', 'drink'])
display('df6', 'df7', 'pd.merge(df6, df7)')

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6, df7)
   name   food drink
0  Mary  bread  wine

> #### **`pd.merge()` + `how=`**

1. `how=outer` > Pone todos los datos por más que no coincidan, los rellena con valores NaN
2. `how=left`
3. `how=right`
   
Aquí hemos fusionado dos conjuntos de datos que **sólo tienen UNA entrada de "nombre" en común**: 'María'.
**Por defecto**, el resultado contiene la **intersección** de los dos conjuntos de entradas; esto es lo que se conoce como **inner join**.

Podemos especificarlo explícitamente mediante la palabra clave ``how``, que por defecto es ``"inner"``:

In [41]:
pd.merge(df6, df7, how='inner') # INNECESARIO = Porque por defecto es inner, no hay que ponerlo

,name,food,drink
0,Mary,bread,wine


> ##### **`how='outer'`** > Rellena los valores perdidos con NaN

In [42]:
display('df6', 'df7', "pd.merge(df6, df7, how='outer')")

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6, df7, how='outer')
     name   food drink
0   Peter   fish   NaN
1    Paul  beans   NaN
2    Mary  bread  wine
3  Joseph    NaN  beer

> ##### **`how='left'`** > Lo que coincide y con los datos no coincidentes del df izquerdo 

In [43]:
display('df6', 'df7', "pd.merge(df6, df7, how='left')")

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6, df7, how='left')
    name   food drink
0  Peter   fish   NaN
1   Paul  beans   NaN
2   Mary  bread  wine

> ##### **`how='right'`** > Lo que coincide y con los datos no coincidentes del df derecho 

In [44]:
display('df6', 'df7', "pd.merge(df6, df7, how='right')")

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6, df7, how='right')
     name   food drink
0    Mary  bread  wine
1  Joseph    NaN  beer

>>> ## **Nombres de columnas solapados: La palabra clave ``suffixes``**
Cuando se unen 2 DataFrames por la columna clave que tienen en común, puede pasar que haya otra columna con mismo nombre pero con distintos datos, por lo que no pueden fusionarse, ya que perdería los datos de una columna. 

Entonces `suffixes=` se usa para **elegir el nombre de las columnas** que son **iguales** pero con **distintos datos** entonces no podemos unir esas columnas.

In [45]:
df8 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [1, 2, 3, 4]})
df9 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [3, 1, 4, 2]})
display('df8', 'df9', 'pd.merge(df8, df9, on="name")')

df8
   name  rank
0   Bob     1
1  Jake     2
2  Lisa     3
3   Sue     4

df9
   name  rank
0   Bob     3
1  Jake     1
2  Lisa     4
3   Sue     2

pd.merge(df8, df9, on="name")
   name  rank_x  rank_y
0   Bob       1       3
1  Jake       2       1
2  Lisa       3       4
3   Sue       4       2

- POR DEFECTO, los sufijos son ``_x`` o ``_y`` para hacer que las columnas tengan distintos nombres.

- PUEDEN ELEGIRSE LOS NOMBRES DE LAS COLUMNAS para que sean mas descriptivos, por lo que, es posible especificar un sufijo personalizado utilizando la palabra clave ``suffixes``:

In [47]:
display('df8', 'df9', 'pd.merge(df8, df9, on="name", suffixes=["_df8", "_df9"])')

df8
   name  rank
0   Bob     1
1  Jake     2
2  Lisa     3
3   Sue     4

df9
   name  rank
0   Bob     3
1  Jake     1
2  Lisa     4
3   Sue     2

pd.merge(df8, df9, on="name", suffixes=["_df8", "_df9"])
   name  rank_df8  rank_df9
0   Bob         1         3
1  Jake         2         1
2  Lisa         3         4
3   Sue         4         2

Estos sufijos funcionan en cualquiera de los posibles patrones de unión, y funcionan también si hay varias columnas superpuestas.

Para más información sobre estos patrones, vea [Aggregation and Grouping](8_Aggregation-and-Grouping.ipynb) donde profundizamos un poco más en el álgebra relacional.
Consulte también la documentación de [Pandas "Merge, Join and Concatenate"](http://pandas.pydata.org/pandas-docs/stable/merging.html) para más información sobre estos temas.

## Ejemplo: Datos de los Estados de EE.UU.

Las operaciones merge y join aparecen con más frecuencia cuando se combinan datos de distintas fuentes.
Aquí consideraremos un ejemplo de algunos datos sobre los estados de EE.UU. y su población.
Los archivos de datos se encuentran en http://github.com/jakevdp/data-USstates/:

In [ ]:
# Los siguientes son comandos shell para descargar los datos
# !curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-population.csv
# !curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-areas.csv
# !curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-abbrevs.csv

Echemos un vistazo a los tres conjuntos de datos, utilizando la función Pandas ``read_csv()``:

In [48]:
pop = pd.read_csv('data/state-population.csv')
areas = pd.read_csv('data/state-areas.csv')
abbrevs = pd.read_csv('data/state-abbrevs.csv')

display('pop.head()', 'areas.head()', 'abbrevs.head()')

pop.head()
  state/region     ages  year  population
0           AL  under18  2012   1117489.0
1           AL    total  2012   4817528.0
2           AL  under18  2010   1130966.0
3           AL    total  2010   4785570.0
4           AL  under18  2011   1125763.0

areas.head()
        state  area (sq. mi)
0     Alabama          52423
1      Alaska         656425
2     Arizona         114006
3    Arkansas          53182
4  California         163707

abbrevs.head()
        state abbreviation
0     Alabama           AL
1      Alaska           AK
2     Arizona           AZ
3    Arkansas           AR
4  California           CA

- Queremos hacer la combinación basándonos en la columna ``state/region`` de ``pop`` y en la columna ``abbreviation`` de ``abbrevs``.

- Utilizaremos ``how='outer'`` para asegurarnos de que no se pierde ningún dato por no coincidir las etiquetas.

- Y eliminaremos la columna `abbreviation` para que no quede redundante, ya que tiene los mismos datos que `state/region` y por eso unimos los DF por ahí.


In [78]:
merged = pd.merge(pop, abbrevs, how='outer',
                  left_on='state/region', right_on='abbreviation')
merged = merged.drop('abbreviation', axis=1) # Elimino columna abbreviation
merged.head()

,state/region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama
2,AL,under18,2010,1130966.0,Alabama
3,AL,total,2010,4785570.0,Alabama
4,AL,under18,2011,1125763.0,Alabama


- Comprobemos si se ha producido alguna discordancia, lo que podemos hacer buscando filas con nulos:

In [79]:
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state            True
dtype: bool

- Algunos de los datos de ``población`` son nulos; ¡averigüemos cuáles son!

In [80]:
merged[merged['population'].isnull()].head()

,state/region,ages,year,population,state
2448,PR,under18,1990,NaN,NaN
2449,PR,total,1990,NaN,NaN
2450,PR,total,1991,NaN,NaN
2451,PR,under18,1991,NaN,NaN
2452,PR,total,1993,NaN,NaN


Parece que todos los valores nulos de población son de Puerto Rico anteriores al año 2000; esto se debe probablemente a que estos datos no están disponibles en la fuente original.

- Y lo que es más importante, vemos también que algunas de las nuevas entradas ``state`` también son nulas, lo que significa que no había ninguna entrada correspondiente en la clave ``abbrevs``.
Averigüemos qué regiones carecen de esta coincidencia:

In [81]:
merged.loc[merged['state'].isnull(), 'state/region'].unique()

array(['PR', 'USA'], dtype=object)

**EXPLICACIÓN SINTAXIS**

`merged.loc[merged['state'].isnull(), 'state/region'].unique()`
- `merged.loc[merged['state'].isnull()` > Muestra en el df merged, los datos nulos de la columna `state`
- ` ,'state/region']` > Los datos nulos de esta columna, no del todo el DF. Lo que mostrará todas las filas donde los datos son nulos. 
- `].unique()` > y acá que muestre solamente en un array los datos de `state/region` donde sean nulos en `state`

- Vemos que los datos nulos estan en la columna 'state', por lo que sabiendo que en 'state/region' las abreviaturas son 'PR' y 'USA' quiere decir Puerto Rico y Estados Unidos, por lo que podemos rellenar la columna 'state' con esos Estados, de la siguiente forma: 

In [82]:
merged.loc[merged['state/region'] == 'PR', 'state'] = 'Puerto Rico'
merged.loc[merged['state/region'] == 'USA', 'state'] = 'United States'
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
dtype: bool

Se acabaron los nulos en la columna ``state``: ¡listo!

- Ahora podemos fusionar el resultado con los datos del área mediante un procedimiento similar.
Al examinar nuestros resultados, querremos unirlos en la columna ``state`` de ambos:

In [129]:
final = pd.merge(merged, areas, on='state', how='left')
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


- De nuevo, vamos a comprobar si hay nulos para ver si hay algún desajuste:

In [102]:
final.isnull().any()

state/region     False
ages             False
year             False
population        True
state            False
area (sq. mi)     True
dtype: bool

- Hay nulos en la columna ``área``; podemos echar un vistazo para ver qué regiones se ignoraron aquí, así:

In [125]:
final['state'][final['area (sq. mi)'].isnull()].unique()

array(['United States'], dtype=object)

- Vemos que nuestro ``DataFrame`` de ``areas`` no contiene el área de Estados Unidos en su conjunto.
Podríamos insertar el valor apropiado (usando la suma de las áreas de todos los estados, por ejemplo), pero en este caso simplemente eliminaremos los valores nulos porque la densidad de población de todo Estados Unidos no es relevante para nuestra discusión actual:

In [127]:
final.dropna(inplace=True)
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


- Ahora tenemos todos los datos que necesitamos. Para responder a la pregunta que nos interesa, vamos a seleccionar primero la parte de los datos correspondiente al año 2000 y la población total.
Usaremos la función ``query()`` para hacerlo rápidamente (esto requiere tener instalado el paquete ``numexpr``; ver [Pandas de alto rendimiento: ``eval()`` y ``query()``](12_Performance-Eval-and-Query.ipynb)):

> #### **``.query()`` = Es una máscara para filtrar con condiciones**
Se utiliza para **filtrar filas** de un DataFrame basado en una expresión booleana.

**SINTAXIS** = `dataframe.query('condición')` >>> Donde `dataframe` es el DataFrame en el que se desea aplicar el filtrado y `'condición'` es una cadena que representa la expresión booleana que se utilizará para filtrar las filas.

**OJO! Con las comillas** > las condiciones van todas entre "" por lo que si tengo nombres tipo string, ponerle comillas simples ''

In [87]:
data2010 = final.query("year == 2010 & ages == 'total'") 
data2010.head()

,state/region,ages,year,population,state,area (sq. mi)
3,AL,total,2010,4785570.0,Alabama,52423.0
91,AK,total,2010,713868.0,Alaska,656425.0
101,AZ,total,2010,6408790.0,Arizona,114006.0
189,AR,total,2010,2922280.0,Arkansas,53182.0
197,CA,total,2010,37333601.0,California,163707.0


El query es lo mismo que hacer una máscara con sintaxis DF, en el ejemplo anterior es más fácil hacer eso que: 
`final[(final['year']==2010) & (final['ages']=='total')]`

- Ahora vamos a calcular la densidad de población y mostrarla por orden.

1. Primero vamos a elegir el índice del DF, que será la columna `state` y, a continuación, calcularemos el resultado:

In [133]:
data2010.columns

Index(['state/region', 'ages', 'year', 'population', 'area (sq. mi)'], dtype='object')

In [ ]:
data2010.set_index('state', inplace=True)

In [136]:
density = data2010['population'] / data2010['area (sq. mi)']

In [141]:
density.sort_values()
density.head()

state
District of Columbia    8898.897059
Puerto Rico             1058.665149
New Jersey              1009.253268
Rhode Island             681.339159
Connecticut              645.600649
dtype: float64

El resultado es una clasificación de los estados de EE.UU. más Washington, DC, y Puerto Rico por orden de densidad de población en 2010, en residentes por milla cuadrada.
Podemos ver que, con diferencia, la región más densa en este conjunto de datos es Washington, DC (es decir, el Distrito de Columbia); entre los estados, el más denso es Nueva Jersey.

También podemos comprobar el final de la lista:

In [142]:
density.tail()

state
South Dakota    10.583512
North Dakota     9.537565
Montana          6.736171
Wyoming          5.768079
Alaska           1.087509
dtype: float64

Vemos que el estado menos denso, con diferencia, es Alaska, con una media de poco más de un residente por milla cuadrada.

Este tipo de fusión de datos desordenados es una tarea habitual cuando se intenta responder a preguntas utilizando fuentes de datos del mundo real.
Espero que este ejemplo te haya dado una idea de las formas en que puedes combinar las herramientas que hemos visto para obtener información de tus datos.